In [445]:
import numpy as np
from numpy import kron, eye as I, diag
import matplotlib.pyplot as plt

from ndgsp.graph.graphs import Graph

from scipy.spatial.distance import pdist, squareform

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

import sys
sys.path.append('..')

from utils.plotting import to_colors


In [307]:
np.random.seed(0)


N = 50
M = 5
lam = 0.4
gamma = 0.5

g = Graph.random_connected(N)


U = g.U

# DG = diag(np.exp(-g.lam))
# DG = I(N)
DG = diag([1] + [0] * (N - 1))

m = 0.8
DS = np.zeros(N)
DS[np.random.choice(np.arange(N), size= int((1 - m) * N))] = 1
DS = diag(DS)

X = np.random.normal(size=(N, M))

lamM, UM = np.linalg.eigh(X.T @ DS @ X)
DM = diag((lamM + lam) ** -0.5)

M11 = DG @ U.T @ DS @ U @ DG + gamma * I(N)
M12 = DG @ U.T @ DS @ X @ UM @ DM

M = np.block([[M11, M12], [M12.T, I(M)]])

lams = np.linalg.eigvalsh(M)

In [311]:
Graph.random_connected(8)

Graph(N=8)

In [422]:
def make_kgr_tikz_graph():
    """
    Create the TikZ code for plotting the KGR diagram
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({-1.5 + offset},2) node{{$t = {t}$}}; \n"
        tex += f"\\draw ({-3.5 + offset},0) node{{ $ \\mathbf{{x}}_{t} = \\begin{{bmatrix}} {x[0]:.2f} \\\\[0.1cm] {x[1]:.2f} \\\\[0.1cm] {x[2]:.2f} \\\\[0.1cm] {x[3]:.2f} \\end{{bmatrix}}$ }};\n"
        tex +=  f"\\draw ({-1.6 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)

    all_tex += '\\draw (1.3, 0) node{$,$}; \n \\draw (8.9, 0) node{$, \\quad\\quad \\dots $};\n\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    return all_tex 

In [ ]:
print(make_kgr_tikz_graph())

In [443]:
def make_rnc_tikz_graph():
    """
    Create the TikZ code for plotting the RNC diagram
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({0 + offset},3) node{{$t = {t}$}}; \n"
        tex +=  f"\\draw ({-2.3 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'

        for xi, yi in [(0.8, 2.3), (-2, 1.2), (0.2, 1.1), (-0.7, -0.4), (1.45, 0), (2, 1.7), (-0.8, -2.3), (-1.6, -1.5), (0.5, -1.8), (2, -1.1)]:

            x_ = np.abs(np.random.normal(size=3))
            tex += f'\\draw ({xi + offset}, {yi}) node{{\\scalebox{{0.7}}{{$[{x_[0]:.1f}, {x_[1]:.1f}, {x_[2]:.1f}]$}}}};\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)


    all_tex += "\draw (2.3, 0) node{$,$};\n" 
    all_tex += '\draw (10.2, 0) node{$, \quad\quad\quad \dots $};\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    return all_tex 

In [ ]:
print(make_rnc_tikz_graph())

In [609]:
np.random.seed(0)
               
N = 12
T = 8
M = 3
gamma = 0.4

X = np.random.normal(size=(T, M))
K = np.exp(-squareform(pdist(X, metric='sqeuclidean')))
lamK, V = np.linalg.eigh(K)


graph = Graph.random_connected(N)
U = graph.U
lamN = graph.lam

# normal
# G = lamK[None, :] ** 0.5 * np.exp(-lamN[:, None])

# SFL
G = lamK[None, :] ** 0.5 * np.array([1] + [0] * (N - 1))[:, None]

# WFL
# G = lamK[None, :] ** 0.5 * np.ones((N, 1))

J = G ** 2 / (G ** 2 + gamma)

DJ = diag(vec(J))
DG = diag(vec(G))

m = 0.6
DS = np.zeros(N * T)
DS[np.random.choice(np.arange(N * T), size=int((1 - m) * N * T), replace=False)] = 1
DS = diag(DS)
DS_ = np.eye(N * T) - DS

In [610]:
def Del(N):
    out = np.zeros((N, N))
    out[0, 0] = 1
    return out

In [611]:
np.kron(V, U) @ DJ @ np.kron(V, U).T

array([[ 0.059,  0.059,  0.059,  0.059,  0.059, ..., -0.   , -0.   , -0.   , -0.   , -0.   ],
       [ 0.059,  0.059,  0.059,  0.059,  0.059, ..., -0.   , -0.   , -0.   , -0.   , -0.   ],
       [ 0.059,  0.059,  0.059,  0.059,  0.059, ..., -0.   , -0.   , -0.   , -0.   , -0.   ],
       [ 0.059,  0.059,  0.059,  0.059,  0.059, ..., -0.   , -0.   , -0.   , -0.   , -0.   ],
       [ 0.059,  0.059,  0.059,  0.059,  0.059, ..., -0.   , -0.   , -0.   , -0.   , -0.   ],
       ...,
       [-0.   , -0.   , -0.   , -0.   , -0.   , ...,  0.056,  0.056,  0.056,  0.056,  0.056],
       [-0.   , -0.   , -0.   , -0.   , -0.   , ...,  0.056,  0.056,  0.056,  0.056,  0.056],
       [-0.   , -0.   , -0.   , -0.   , -0.   , ...,  0.056,  0.056,  0.056,  0.056,  0.056],
       [-0.   , -0.   , -0.   , -0.   , -0.   , ...,  0.056,  0.056,  0.056,  0.056,  0.056],
       [-0.   , -0.   , -0.   , -0.   , -0.   , ...,  0.056,  0.056,  0.056,  0.056,  0.056]])

In [628]:
-np.sort(-np.real(np.linalg.eigvals(np.kron(V @ diag(lamK / (lamK + gamma)) @ V.T, np.eye(N)) @ DS_)))

array([0.805, 0.785, 0.785, 0.785, 0.783, 0.782, 0.781, 0.765, 0.74 , 0.74 , 0.724, 0.724, 0.717, 0.717, 0.715, 0.715, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.714, 0.704, 0.704, 0.696, 0.692, 0.692, 0.688, 0.683, 0.681, 0.68 , 0.672, 0.672, 0.636, 0.635, 0.635, 0.597, 0.58 , 0.579, 0.573, 0.572, 0.571, 0.554, 0.554, 0.547, 0.547, 0.547, 0.546, 0.424, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ])

In [634]:
-np.sort(-np.real(np.linalg.eigvals(N ** -1 * np.kron(V @ diag(lamK / (lamK + gamma)) @ V.T, np.ones((N, N))) @ DS_)))

array([ 0.536,  0.536,  0.476,  0.476,  0.417,  0.417,  0.357,  0.238,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ])

In [644]:
-np.sort(-np.real(np.linalg.eigvals(T ** -1 * N ** -1 * np.ones((N*T, N*T)) @ DS_)))

array([ 0.604,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   ])

In [642]:
np.linalg.eigvals(np.ones((N*T, N*T)) @ DS_)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 58., -0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [643]:
T ** -1 * N ** -1 * DS_.sum() * 1 / (1 + gamma)

0.43154761904761907

In [645]:
T ** -1 * N ** -1 *DS_.sum()

0.6041666666666666

In [618]:
lamK / (lamK + gamma)

array([0.402, 0.572, 0.688, 0.707, 0.714, 0.714, 0.775, 0.831])

In [615]:
np.linalg.eigvals(np.ones((N, N)))

array([ 0., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [592]:
lam_sim = np.linalg.eigvals(np.kron(V, U) @ DJ @ np.kron(V, U).T @ DS_)

print(np.abs(lam_sim).max())
print(lamK.max() / (lamK.max() + gamma))

0.8312180881242061
0.8312180881242064


In [593]:
lam_cgm = np.linalg.eigvalsh(DG @ np.kron(V, U).T @ DS @ np.kron(V, U) @ DG + gamma * I(N * T))

print(lam_cgm.max() / lam_cgm.min())
print((lamK.max() + gamma) / gamma)

3.500000000000005
5.924805501290318
